In [16]:
import torch
from torchvision import transforms
import pandas as pd
import os
import torch.nn as nn
from torch.utils.data import Dataset


In [3]:
ROOT_DIR=R'C:\Users\User\Downloads\60 days of python\day-33'
DATA_DIR=os.path.join(ROOT_DIR,'Data')

In [4]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [5]:
dataset_path=os.path.join(DATA_DIR,'digit_train.csv')

In [6]:
data=pd.read_csv(dataset_path)
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [8]:
pixels=data.iloc[0].values[1:].astype('float32')
label=int(data.iloc[0].values[0])
print(pixels.shape)
print(label)

(784,)
1


In [9]:
pixel=torch.tensor(pixels)
print(pixel[210:220])

tensor([  0.,   0.,   0.,  80., 247., 253., 208.,  13.,   0.,   0.])


In [10]:
pixel.shape

torch.Size([784])

In [13]:
pixels=pixel.reshape(-1,28,28)
print(pixels)

tensor([[[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 188., 255.,
           94.,   0.,   0.,   0.,   0.,   0.],
     

In [14]:
pixels_transformation = transforms.Compose([
    transforms.Normalize(
        mean=torch.tensor([0.1307]),
        std=torch.tensor([0.3081])),
])
pixels = pixels_transformation(pixels)

In [17]:
class DigitDataset(Dataset):
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path)
        print(data.head())
        self.transform = transform

    def __len__(self):
        """Returns the total number of examples in the dataset"""
        return len(self.data)

    def __getitem__(self, idx):
        """ Focus to process a single example in the dataset """
        pixels = self.data.iloc[idx].values[1:].astype('float32')
        label = int(self.data.iloc[idx].values[0])

        pixels = torch.tensor(pixels)
        label = torch.tensor(label)

        pixels = pixels.reshape(28, 28).unsqueeze(0) / 255.0
        if self.transform:
            pixels = pixels_transformation(pixels)

        return pixels, label

In [20]:
dataset=DigitDataset(dataset_path,transform=True)

   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   
3       0  ...         0         0         0         0         0         0   
4       0  ...         0         0         0         0         0         0   

   pixel780  pixel781  pixel782  pixel783  
0         0         0         

# **splitting the dataset into train,validation and testing datasets**

In [21]:
from torch.utils.data import random_split

torch.manual_seed(42)

train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset=dataset,
    lengths=[train_size, val_size, test_size],

)

In [24]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    shuffle=True,
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=32,
    shuffle=False,
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=32,
    shuffle=False,
)

# **Model**

In [25]:
class DigitClassifier(nn.Module):
    def __init__(self):
        super(DigitClassifier, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        return x

In [26]:
model=DigitClassifier().to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=0.001)


In [27]:
checkpoint_path=os.path.join(
    os.getcwd(),"checkpoint","best_model.pth"
)

In [28]:
if os.path.exists(checkpoint_path):
    model.load_state_dict(torch.load(checkpoint_path))
    print("Model Loaded Successfully from  checkpoints")

In [34]:
def train_model(
        model,
        train_loader, val_loader,
        criterion, optimizer,
        num_epochs,
        device,
        checkpoint_path
):
    model.train()

    best_val_accuracy = 0
    patience = 3

    # Checkpoints
    checkpoint_dir = os.path.dirname(checkpoint_path)
    """checkpoints means model ta onekkhn run korar por jodi suddenly computer ta off kore dei then jodi abr model ta start korle se jototuku porjnto trained hoise tototuku thekei jeno abr train howa suru kore...Infact model ta jate abr zero theke train na howa suru kore """
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    for epoch in range(num_epochs):
        train_loss = 0
        for batch_idx, (pixel_batch, label_batch) in enumerate(train_loader):
            pixel_batch, label_batch = pixel_batch.to(device), label_batch.to(device)

            optimizer.zero_grad()
            output = model(pixel_batch)
            loss = criterion(output, label_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader.dataset)
        val_loss, val_accuracy = evaluate_model(
            model=model,
            data_loader=val_loader,
            criterion=criterion,
            device=device
        )

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), checkpoint_path)
            print("Best parameter so far.")
        else:
            """ early stopping """
            patience = patience - 1

        print(f"epoch [{epoch+1}/{num_epochs}], loss: {train_loss:.4f}", end=" ")
        print(f"val_loss: {val_loss:.4f}", end=" ")
        print(f"val_acc: {val_accuracy:.4f}")

        if patience == 0:
            """ Callback: Early stopping """
            print(f"Model performance is not improving. Exiting the training.")
            break

In [32]:
def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    val_loss = 0
    accuracy = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item()

            _, predicted = torch.max(output.data, 1)
            accuracy += (predicted == target).sum().item()

        val_loss /= len(data_loader.dataset)
        accuracy /= len(data_loader.dataset)
    return val_loss, accuracy

In [35]:
train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=100,
    device=device,
    checkpoint_path=checkpoint_path,
)

Best parameter so far.
epoch [1/100], loss: 0.0098 val_loss: 0.0059 val_acc: 0.9410
Best parameter so far.
epoch [2/100], loss: 0.0043 val_loss: 0.0041 val_acc: 0.9608
Best parameter so far.
epoch [3/100], loss: 0.0030 val_loss: 0.0030 val_acc: 0.9695
epoch [4/100], loss: 0.0022 val_loss: 0.0032 val_acc: 0.9675
Best parameter so far.
epoch [5/100], loss: 0.0019 val_loss: 0.0031 val_acc: 0.9727
epoch [6/100], loss: 0.0015 val_loss: 0.0033 val_acc: 0.9698
Best parameter so far.
epoch [7/100], loss: 0.0012 val_loss: 0.0028 val_acc: 0.9744
Best parameter so far.
epoch [8/100], loss: 0.0011 val_loss: 0.0030 val_acc: 0.9757
epoch [9/100], loss: 0.0009 val_loss: 0.0030 val_acc: 0.9751
Model performance is not improving. Exiting the training.


In [36]:
model.load_state_dict(torch.load(checkpoint_path))

val_loss, val_acc = evaluate_model(
    model=model,
    data_loader=val_loader,
    criterion=criterion,
    device=device
)

print(f"test_loss: {val_loss:0.4f}, test_acc: {val_acc:0.4f}")

test_loss: 0.0030, test_acc: 0.9757


In [37]:
test_loss, test_accuracy = evaluate_model(
    model=model,
    data_loader=test_loader,
    criterion=criterion,
    device=device
)

print(f"test_loss: {test_loss:0.4f}, test_acc: {test_accuracy:0.4f}")

test_loss: 0.0044, test_acc: 0.9657


In [38]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

X_train = np.array([data[0].numpy().flatten() for data in train_dataset])
y_train = np.array([data[1] for data in train_dataset])
X_test = np.array([data[0].numpy().flatten() for data in test_dataset])
y_test = np.array([data[1] for data in test_dataset])

dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, y_train)

y_pred = dt_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'DecisionTreeClassifier test accuracy: {accuracy:.2f}')

DecisionTreeClassifier test accuracy: 0.85
